<a href="https://colab.research.google.com/github/froggagul/nsmc/blob/master/Naver_sentiment_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download raw datasets
import requests


f_train = requests.get('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
f_test = requests.get('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

open('train.txt', 'wb').write(f_train.content)
open('test.txt', 'wb').write(f_test.content)

4893335

In [1]:
!pip install transformers

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertModel, AdamW
from tqdm.notebook import tqdm

# reproducible pytorch
torch.manual_seed(1234)

In [3]:
class NSMCDataset(Dataset):
  
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path, sep='\t')
        
        # drop nan row
        self.dataset = self.dataset.dropna(axis = 0)
        # drop duplicate row
        self.dataset['document'] = self.dataset['document'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
        self.dataset.drop_duplicates(subset=['document'], inplace=True)
        
        # tokenizer
        self.tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
  
        # add special tokens
        self.tokenizer.add_tokens(["OO", "OOO", "OOOO"], special_tokens=True)
        
    def __len__(self):
        return len(self.dataset)
  
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 1:3].values # no ids!
    
        '''
        ["document", "label"]
        '''
        document = row[0]
        label = row[1]

        inputs = self.tokenizer(
            document, 
            return_tensors='pt',
            truncation=True,
            max_length=256,
            pad_to_max_length=True,
            add_special_tokens=True
            )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, label

In [4]:
train_dataset = NSMCDataset("train.txt")
test_dataset = NSMCDataset("test.txt")

In [5]:
class BertClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertModel.from_pretrained("kykim/bert-kor-base")
        self.fc = nn.Linear(768, 2)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids = input_ids, attention_mask = attention_mask)

        pooled_output = outputs[1]
        return self.fc(pooled_output)

In [6]:
device = torch.device("cuda")
model = BertClassifier()

model.to(device)

model.bert_model.resize_token_embeddings(len(train_dataset.tokenizer))

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(42003, 768)

In [7]:
epochs = 4
batch_size = 32

In [8]:
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [9]:
losses = []
accuracies = []

for epoch in range(1, epochs + 1):
    total_loss = 0.0
    correct = 0
    total = 0
    batch_index = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()

        y_batch = y_batch.to(device)
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        
        y_pred = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)
        loss = F.cross_entropy(y_pred, y_batch)

        loss.backward()

        optimizer.step()

        total_loss += loss.item()

        predicted = torch.max(y_pred, 1)[1]
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batch_index += 1
        if batch_index % 500 == 0:
            print(f"epoch #{epoch} {batch_index} Batch Loss:{total_loss} Accuracy:{correct.float() / total}")

    losses.append(total_loss)
    accuracies.append(correct.float() / total)
    print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)
    torch.save(model.state_dict(), f"model_{epoch}_v2.pt")

  0%|          | 0/4569 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


epoch #1 500 Batch Loss:182.74187215417624 Accuracy:0.8345625400543213
epoch #1 1000 Batch Loss:328.26211074739695 Accuracy:0.8566563129425049
epoch #1 1500 Batch Loss:462.2294279932976 Accuracy:0.867354154586792
epoch #1 2000 Batch Loss:589.996418595314 Accuracy:0.8742969036102295
epoch #1 2500 Batch Loss:715.1320393458009 Accuracy:0.8788874745368958
epoch #1 3000 Batch Loss:840.2887992821634 Accuracy:0.8817812204360962
epoch #1 3500 Batch Loss:965.05376829952 Accuracy:0.8841696381568909
epoch #1 4000 Batch Loss:1086.0951257683337 Accuracy:0.8859140872955322
epoch #1 4500 Batch Loss:1210.6286521218717 Accuracy:0.887131929397583
Train Loss: 1226.6280853841454 Accuracy: tensor(0.8873, device='cuda:0')


  0%|          | 0/4569 [00:00<?, ?it/s]

epoch #2 500 Batch Loss:97.68372974172235 Accuracy:0.9228750467300415
epoch #2 1000 Batch Loss:194.7932618074119 Accuracy:0.9223437905311584
epoch #2 1500 Batch Loss:290.0116873383522 Accuracy:0.9230416417121887
epoch #2 2000 Batch Loss:391.89438582398 Accuracy:0.9218125343322754
epoch #2 2500 Batch Loss:488.95884189754725 Accuracy:0.9221874475479126
epoch #2 3000 Batch Loss:583.1151387579739 Accuracy:0.922989547252655
epoch #2 3500 Batch Loss:684.7481634691358 Accuracy:0.9223392605781555
epoch #2 4000 Batch Loss:783.0311699621379 Accuracy:0.9222187995910645
epoch #2 4500 Batch Loss:882.584127008915 Accuracy:0.921999990940094
Train Loss: 896.1151333004236 Accuracy: tensor(0.9220, device='cuda:0')


  0%|          | 0/4569 [00:00<?, ?it/s]

epoch #3 500 Batch Loss:73.84007373359054 Accuracy:0.9442500472068787
epoch #3 1000 Batch Loss:148.66366141755134 Accuracy:0.9432500600814819
epoch #3 1500 Batch Loss:224.65588566008955 Accuracy:0.9433124661445618
epoch #3 2000 Batch Loss:298.53374948492274 Accuracy:0.9431719183921814
epoch #3 2500 Batch Loss:373.9076549853198 Accuracy:0.942674994468689
epoch #3 3000 Batch Loss:451.8605559174903 Accuracy:0.9422603845596313
epoch #3 3500 Batch Loss:524.4185101347975 Accuracy:0.9426785707473755
epoch #3 4000 Batch Loss:599.1007118965499 Accuracy:0.9427812695503235
epoch #3 4500 Batch Loss:672.722677425947 Accuracy:0.9429236054420471
Train Loss: 682.3519245530479 Accuracy: tensor(0.9430, device='cuda:0')


  0%|          | 0/4569 [00:00<?, ?it/s]

epoch #4 500 Batch Loss:51.62225419934839 Accuracy:0.9619375467300415
epoch #4 1000 Batch Loss:104.21161222271621 Accuracy:0.9609062671661377
epoch #4 1500 Batch Loss:157.67967288731597 Accuracy:0.9606249928474426
epoch #4 2000 Batch Loss:213.2837757037487 Accuracy:0.9597344398498535
epoch #4 2500 Batch Loss:273.6601978566032 Accuracy:0.9587875008583069
epoch #4 3000 Batch Loss:327.48131005535834 Accuracy:0.9591978788375854
epoch #4 3500 Batch Loss:383.856889689574 Accuracy:0.9591249823570251
epoch #4 4000 Batch Loss:439.17187310312875 Accuracy:0.9591328501701355
epoch #4 4500 Batch Loss:495.52115909778513 Accuracy:0.9588680863380432
Train Loss: 502.30119440332055 Accuracy: tensor(0.9589, device='cuda:0')


In [10]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))
    predicted = torch.max(y_pred, 1)[1]
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/6145 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Accuracy: tensor(0.9111, device='cuda:0')


In [11]:
losses, accuracies

([1226.6280853841454,
  896.1151333004236,
  682.3519245530479,
  502.30119440332055],
 [tensor(0.8873, device='cuda:0'),
  tensor(0.9220, device='cuda:0'),
  tensor(0.9430, device='cuda:0'),
  tensor(0.9589, device='cuda:0')])

In [11]:
!nvidia-smi

Sun Sep 19 03:00:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    42W / 250W |  11575MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------